### 다중분류

`LogisticRegression(C=20, max_iter=1000)`
- `C` 는 L2 규제(릿지?) 적용한다는 말로 기본으로 1 
  선형회귀는 강도는 alpha인데 선형회귀랑 반대로 C값이 내려가야지 규제가 강해짐
- `max_iter` 로지스틱은 반복계산, 반복 획쉬 지정을 하는데 기본으로 100임


In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import numpy as np


fish = pd.read_csv('https://bit.ly/fish_csv_data')

# Boolean indexing 으로 조건문으로 가지고 true false 해서 indexing하기
char_arr = np.array(['A','B','C','D','E'])
bool_index = char_arr[[True,False,True,False,False]]
bool_index

# 도미 빙어 적용
fish_input = fish[['Weight', 'Length','Diagonal','Height', 'Width']].to_numpy()
fish_target = fish['Species'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(
  fish_input, fish_target, random_state=42)

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

# 다중분류 7가지 생성 분류
lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled, train_target)

print(lr.score(train_scaled, train_target), lr.score(test_scaled, test_target))

proba = lr.predict_proba(test_scaled[:5])
print(lr.classes_)
print(np.round(proba, decimals =3))
# Perch
# Smelt
# Pike
# Roach
# Perch

print(f'기울기 :{lr.coef_.shape}, 절편 : {lr.intercept_.shape}' )
# 기울기 절편 모두 z값을 7개 계산
# 다중 분류는 클래스별로 선형함수 z값을 계산

lr.coef_, lr.intercept_
# z값은 무한대로 확률을 위해 7가지 z값 총합을 0~1사이로 변경이 필요하다 -> 소프트맥스

0.9327731092436975 0.925
['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']
[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]
기울기 :(7, 5), 절편 : (7,)


(array([[-1.49001999, -1.02912482,  2.59345218,  7.70357843, -1.2007022 ],
        [ 0.19618201, -2.01068627, -3.77976461,  6.50491598, -1.99482485],
        [ 3.56279819,  6.34356926, -8.48970993, -5.757574  ,  3.79307177],
        [-0.10458121,  3.603196  ,  3.93067836, -3.61737079, -1.75069674],
        [-1.40061524, -6.07503319,  5.25969396, -0.87219889,  1.86043687],
        [-1.38526189,  1.49214385,  1.39226012, -5.67734222, -4.40097623],
        [ 0.62149812, -2.32406483, -0.9066101 ,  1.71599149,  3.69369137]]),
 array([-0.09205181, -0.26290863,  3.25101315, -0.14742819,  2.65498377,
        -6.78783248,  1.38422419]))

### 소프트맥스(Sofrmax)
- 소프트맥스는 총합이 1인 형태로 바꿔서 계산해 주는 함수
- 합계가 1인 형태로 변환하면 클 값이 두드러지게 나타나고, 작은 값은 더 작아짐
- 이 값이 교차 엔트로피를 지나 [1.,0.,0.]으로 변화하게 되면 우리가 원하는 원-핫-인코딩값 즉 하나만 1이고 나머지는 모두 0인 형태로 전환

  <!-- ![alt text](소프트맥스.png) -->
  <img src="소프트맥스.png" alt="drawing" width="500"/>
- 타겟이 7가지며 7개의 z 값을 계산해 갖고 있음(가장 큰 z값이 분류 결정)
- 이를 확률 7개 합이 1이 되도록 변환 필요
- 소프트 맥스 함수 이용

In [13]:
# 소프트맥스
# z값을 계산하여 소프트 맥스로 변환 z1 ~ z7까지 구함

# 앞에서 5개만 decision_function으로 z값 계산
decisions = lr.decision_function(test_scaled[:5])

# 7개의 z값이 나옴
print(np.round(decisions, decimals=2),'\n')

# z값을 소프트 맥스함수로 합이 1이 되도록 변환
from scipy.special import softmax

# 각 행별로 소프트 맥스 계산
proba = softmax(decisions, axis=1) 
# proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))

[[ -6.5    1.03   5.16  -2.73   3.34   0.33  -0.63]
 [-10.86   1.93   4.77  -2.4    2.98   7.84  -4.26]
 [ -4.34  -6.23   3.17   6.49   2.36   2.42  -3.87]
 [ -0.68   0.45   2.65  -1.19   3.26  -5.75   1.26]
 [ -6.4   -1.99   5.82  -0.11   3.5   -0.11  -0.71]] 

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]


### 결론
- 분류모델은 예측 뿐만 아니라 예측의 근거가 되는 확률을 출력 가능
- 확률은 분류 모델이 얼마나 예측을 확신하는지를 나타냄
- KNN은 확률을 출력할 수 있지만 샘플 클래스의 비율 -> 항상 정해진 확률만 출력 -> 로지스틱은 이런 한계점을 극복
- 선형회귀처럼 계산한 값을 그래도 출력하는 것이 아닌 이 값을 0~1의 사이로 압축 = 0~100%사이의 확률

이진분류
- 하나의 선형 방정식을 훈련
- 방정식의 출력값을 시그모이트 함수를 이용하여 0~1사이의 값을 만듦
- 이 값이 양성 레이블에 대한 확률이 됨(음성은 1-양성)

다중분류
- 클래스의 개수만큼 방정식을 훈련
- 각 방정식의 출력 값을 소프트맥스 함수를 이용해 전체 클래스(레이블)에 대한 합이 1이 되도록 만듦
- 이 값이 각 레이블에 대한 확률이 됨